In [1]:
!pip install crewai groq --progress-bar off
!sudo apt install tesseract-ocr --progress-bar off
!sudo apt install libtesseract-dev --progress-bar off
!pip install pytesseract --progress-bar off

E: Command line option --progress-bar is not understood in combination with the other options
E: Command line option --progress-bar is not understood in combination with the other options


In [20]:
!sudo apt install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,023 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [4]:
!export GROQ_API_KEY="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"


In [2]:
import os
os.environ['GROQ_API_KEY'] = "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"
os.environ["REPLICATE_API_TOKEN"] = "r8_uCrjLA7XnQEl5N322zN54Rxtk2xY6287jT19F"


In [3]:
from crewai import Agent, Task, Crew, LLM
import io
from PIL import Image
import pytesseract

In [15]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union
import numpy as np
np.float_ = np.float64
import pandas as pd
from crewai import Agent, Crew, Process, Task
from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult


import io
from PIL import Image
import pytesseract


In [9]:
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=1,
    max_completion_tokens=1024,
    top_p=0.9,
    stop=None,
    stream=False,
)

import shutil


In [21]:
def run_tesseract(img):

    # img = Image.open(img)
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='JPEG')
    img_byte_arr = img_byte_arr.getvalue()
    img = Image.open(io.BytesIO(img_byte_arr))

    pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
    text = pytesseract.image_to_string(img)
    # del state['the_image']
    return {text}
img = Image.open("/content/airbnb.png").convert("RGB")
text = run_tesseract(img)

In [22]:
from crewai import Agent, Task, Crew, LLM

# Initialize Large Language Model (LLM) of your choice (see all models on our Models page)
# llm = LLM(model="groq/llama-3.1-70b-versatile")

# Create your CrewAI agents with role, main goal/objective, and backstory/personality
summarizer = Agent(
    role='Receipt Price Extractor', # Agent's job title/function
    goal='Take out the total price out of the text and remove all currency signs', # Agent's main objective
    backstory='Technical price extractor who excels at identifying price from receipts', # Agent's background/expertise
    llm=llm, # LLM that powers your agent
    verbose=True # Show agent's thought process as it completes its task
)

translator = Agent(
    role='Receipt categorizer',
    goal='categorizer Receipts to different types like fuel, travel, hotel, food etc.',
    backstory='Technical Receipt categorizer specializing in identifying type of receipts',
    llm=llm,
    verbose=True
)

# Define your agents' tasks
summary_task = Task(
    description=f"Extract the price from the text: {text}",
    expected_output="Return me just the price as a number",
    agent=summarizer # Agent assigned to task
)

translation_task = Task(
    description=f"Extract the receipt type from the text:  {text}",
    expected_output="just categorize the text in one word",
    agent=translator,
    dependencies=[summary_task] # Must run after the summary task
)

# Create crew to manage agents and task workflow
crew = Crew(
    agents=[summarizer, translator], # Agents to include in your crew
    tasks=[summary_task, translation_task], # Tasks in execution order
    verbose=True
)

result = crew.kickoff()
print(result)



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9066701a-24fb-4b98-bcf1-6be8f30fde1a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 64c44d0c-d69e-48e7-9658-8df7f384eb99
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 64c44d0c-d69e-48e7-9658-8df7f384eb99
       Status: Executing Task...
    └── 🤖 Agent: Receipt Price Extractor
            Status: In Progress

# Agent: Receipt Price Extractor
## Task: Extract the price from the text: {'OPEN 2s @\n\n12:46 all FS ©\n\n10f 1\n\nPrice breakdown\n\n26,500.00 x 4 nights 226,000.00\nLong stay discount -21,300.00\nCleaning fee 1,000.00\nTaxes and fees 23,084.00\nTotal (INR) 228,784.00\nPayment\n\nVISA ++ 5359 28,784.00\n\nFebruary 26, 2025, 10:21:23 AM GMT+5:30\n\nAmount paid (INR) 28,784.00\n\nment of the Total Price to Airbnb Payments, your payment obligation to your\n\n) policy (available on the Listing); or (ji) Rebocking and Refund Policy Tecms,\n\n \n\nLOOKS TOOLS EXPORT\n\x0c'}


🚀 Crew: crew
└── 📋 Task: 64c44d0c-d69e-48e7-9658-8df7f384eb99
       Status: Executing Task...
    └── 🤖 Agent: Receipt Price Extractor
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 64c44d0c-d69e-48e7-9658-8df7f384eb99
│      Assigned to: Receipt Price Extractor
│      Status: ✅ Completed
│   └── 🤖 Agent: Receipt Price Extractor
│           Status: ✅ Completed
└── 📋 Task: 90aa8e6f-2b14-4181-aff6-0d2ccb91a9b6
       Status: Executing Task...
    └── 🤖 Agent: Receipt categorizer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 64c44d0c-d69e-48e7-9658-8df7f384eb99
│      Assigned to: Receipt Price Extractor
│      Status: ✅ Completed
│   └── 🤖 Agent: Receipt Price Extractor
│           Status: ✅ Completed
└── 📋 Task: 90aa8e6f-2b14-4181-aff6-0d2ccb91a9b6
       Assigned to: Receipt categorizer
       Status: ✅ Completed
    └── 🤖 Agent: Receipt categorizer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 90aa8e6f-2b14-4181-aff6-0d2ccb91a9b6                                                                     │
│  Agent: Receipt categorizer                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9066701a-24fb-4b98-bcf1-6be8f30fde1a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Hotel
